<a href="https://colab.research.google.com/github/tonyhoo/ML-000/blob/main/Week2_Homework-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%load_ext Cython


## Original Version
* Benchmark result around 2 second

In [4]:
import numpy as np
import pandas as pd


def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result



y = np.random.randint(2, size=(500, 1))
x = np.random.randint(10, size=(500, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])


In [66]:
%%timeit

result = target_mean_v1(data, 'y', 'x')
print(result[0:10])

[0.54       0.54       0.56       0.5        0.54       0.46666667
 0.48275862 0.54       0.42222222 0.59259259]
[0.54       0.54       0.56       0.5        0.54       0.46666667
 0.48275862 0.54       0.42222222 0.59259259]
[0.54       0.54       0.56       0.5        0.54       0.46666667
 0.48275862 0.54       0.42222222 0.59259259]
[0.54       0.54       0.56       0.5        0.54       0.46666667
 0.48275862 0.54       0.42222222 0.59259259]
1 loop, best of 3: 2.31 s per loop


## Verion 1 - Replace `pandas.groupby`
* Benchmark result around 19.1 ms

In [8]:
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    dict_sum = {}
    dict_count = {}
    for i in range(data.shape[0]):
      x_value = data.loc[i, x_name]
      if x_value in dict_sum.keys():
        dict_sum[x_value] += data.loc[i, y_name]
        dict_count[x_value] += 1
      else:
        dict_sum[x_value] = data.loc[i, y_name]
        dict_count[x_value] =1 

    for i in range(data.shape[0]):
      result[i] = (dict_sum[data.loc[i, x_name]] - data.loc[i, y_name])/(dict_count[data.loc[i, x_name]]-1)

    return result



In [9]:
%%timeit

result = target_mean_v2(data, 'y', 'x')


10 loops, best of 3: 19.1 ms per loop


## Version 2 - Cython with same Python codes
* Benchmark result around 18.9 ms

In [32]:
%%cython -a

import numpy as np


def target_mean_v3(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    dict_sum = {}
    dict_count = {}
    for i in range(data.shape[0]):
      x_value = data.loc[i, x_name]
      if x_value in dict_sum.keys():
        dict_sum[x_value] += data.loc[i, y_name]
        dict_count[x_value] += 1
      else:
        dict_sum[x_value] = data.loc[i, y_name]
        dict_count[x_value] =1 

    for i in range(data.shape[0]):
      result[i] = (dict_sum[data.loc[i, x_name]] - data.loc[i, y_name])/(dict_count[data.loc[i, x_name]]-1)

    return result



In [34]:
%%timeit
result = target_mean_v3(data, 'y', 'x')


10 loops, best of 3: 18.9 ms per loop


## Version 3 - Cython with typed codes
* Benchmark result around 14 us

In [72]:
%%cython -a

import numpy as np
cimport numpy as cnp


cpdef target_mean_v4(cnp.ndarray data, int y_index, int x_index):
    cdef cnp.ndarray result
    cdef int data_length
    dict_sum = {}
    dict_count = {}

    data_length = data[0].shape[0]
    result = np.zeros(data_length)
    for i in range(data_length):
      x_value = data[x_index][i]
      if x_value in dict_sum.keys():
        dict_sum[x_value] += data[y_index][i]
        dict_count[x_value] += 1
      else:
        dict_sum[x_value] = data[y_index][i]
        dict_count[x_value] =1 

    for i in range(data_length):
      result[i] = (dict_sum[data[x_index][i]] - data[y_index][i])/(dict_count[data[x_index][i]]-1)

    return result



In [73]:
%%timeit

result = target_mean_v4(data.to_numpy(), 0, 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in long_scalars
  


The slowest run took 45.72 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 13.9 µs per loop


array([0.54      , 0.54      , 0.56      , 0.5       , 0.54      ,
       0.46666667, 0.48275862, 0.54      , 0.42222222, 0.59259259])